# PART 3 - NER STREAMLIT WEB UI

Lastly, we want to create a streamlit app in order for the end-user to have a platform that he can easily upload a .txt file and see the entities and their respective classes that the model identified.

We'll define a python script with streamlit code. I will only copy it here in order to better organize the Jupyter Notebook, but as we did with part 2, the full .py script is in the folder, because we'll want to run it from a shell command after.

In [ ]:
import streamlit as st
import pandas as pd
import requests

DOCKER_CONTAINER_URL = "http://localhost:8000/ner"

# Align logo and user instructions using streamlit columns
col1, col2, col3 = st.columns([1, 2, 1])
with col2:  # center col
    st.image("sleek_logo.png", width=400)

    st.title("NER App")

    st.write("Welcome to Sleek's NER App!")
    st.write("")
    st.write("Upload a text file to extract named entities:")

    # File upload widget
    uploaded_file = st.file_uploader(label='Choose text file', type="txt")

if uploaded_file is not None:

    file_contents = uploaded_file.read().decode("utf-8")

    # Limiting display of the file contents
    max_display_length = 2000
    if len(file_contents) > max_display_length:
        st.write("File content (truncated for display):")
        st.write(file_contents[:max_display_length] + "...")
    else:
        st.write("File content:")
        st.write(file_contents)

    # Sending text to the NER service running in the Docker container
    try:
        response = requests.post(
            DOCKER_CONTAINER_URL,
            json={"text": file_contents}
        )
        # Raising error if the request unsuccessful
        response.raise_for_status()

        ner_results = response.json()

        # Checking if there are entities in the response
        if ner_results and "entities" in ner_results:
            data = {
                "Entity Class": [entity["entity_group"] for entity in ner_results["entities"]],
                "Entity": [entity["word"] for entity in ner_results["entities"]]
            }

            # Displaying as table
            df = pd.DataFrame(data)
            st.write("Entities Identified:")
            st.table(df)
        else:
            st.write("No entities found :(")

    except requests.exceptions.RequestException as e:
        st.error(f"Error during NER processing: {e}")

# Footer
st.write("---")
st.write("Made by Sergio Bank for Sleek")


The final step is to run our streamlit app! - in dark mode of course, because it's sleeker ;)

In [ ]:
! streamlit run ner_streamlit_app.py --theme.base dark


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.110:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


It is accessible on http://localhost:8501

There is a 'example_sentences.txt' file with some sentences that I tried uploading, and here's the result:

![title](result_screenshot_1.png)
![title](result_screenshot_2.png)